In [43]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
#from gensim import corpora, models, similarities 

time: 2.59 ms


In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 732 ms


In [45]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.31 s


In [46]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 1.18 ms


[autoreload of autotime failed: Traceback (most recent call last):
  File "/home/jshoun01/.virtualenvs/py27/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


In [47]:
ls data

data@
time: 546 ms


In [48]:
data_dir='data'

time: 567 ms


In [49]:

posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'),max_num_children=5000)

#posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'))

time: 36.2 s


In [50]:
posts_df.columns

Index([u'AcceptedAnswerId', u'AnswerCount', u'Body', u'ClosedDate',
       u'CommentCount', u'CommunityOwnedDate', u'CreationDate',
       u'FavoriteCount', u'Id', u'LastActivityDate', u'LastEditDate',
       u'LastEditorDisplayName', u'LastEditorUserId', u'OwnerDisplayName',
       u'OwnerUserId', u'ParentId', u'PostTypeId', u'Score', u'Tags', u'Title',
       u'ViewCount'],
      dtype='object')

time: 2.27 ms


In [51]:
posts_df.irow(0).Body

u'<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes t

time: 529 ms


In [52]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,NaN,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,NaN,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN


time: 718 ms


Only interested in questions

In [53]:
print("There are %s posts"%(len(posts_df)))
posts_df=posts_df[posts_df.PostTypeId=="1"]
print("There are %s question posts"%(len(posts_df)))

There are 5001 posts
There are 1293 question posts
time: 517 ms


## Process Text

In [54]:
# strip html tags
posts_df['BodyCleaned']=posts_df['Body'].apply(lambda x : helper.strip_tags(x))
# tokenize text
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.tokenize_and_stem(x,None))

# remove stop words
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.remove_stopwords(x))



time: 4.96 s


In [55]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,BodyCleaned
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584,"[A, Vista, virtual, machine, I, use, GB, virtu..."
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654,"[I, used, record, important, technical, meetin..."
5,42,9,<p>What is the difference between the two supe...,NaN,0,NaN,2009-07-15T07:11:46.370,1,29,2011-11-25T19:11:50.960,...,48078,NaN,13,NaN,1,17,<linux><unix><sudo>,"What's the difference between the commands ""su...",14700,"[What, difference, two, super, user, commands,..."
7,3841,3,"<p>My late 2008, Macbook Pro Unibody machine h...",NaN,2,NaN,2009-07-15T07:17:13.970,1,37,2009-07-15T21:15:21.323,...,NaN,NaN,26,NaN,1,4,<mac><crash><boot-camp>,Why does the Macbook Pro Unibody crash on hibe...,1169,"[My, late, Macbook, Pro, Unibody, machine, num..."
13,92,2,<p>Is there any way to configure application t...,NaN,0,NaN,2009-07-15T07:20:00.717,1,46,2014-07-26T13:37:09.967,...,81403,NaN,21,NaN,1,9,<windows-vista><boot><uac><privileges><elevation>,Elevated Priviliges for Startup Applications i...,7997,"[Is, way, configure, application, requires, ad..."


time: 36.7 ms


In [56]:
len(posts_df)

1293

time: 557 ms


In [57]:
texts = posts_df['BodyCleaned']

time: 736 ms


In [58]:
#create a Gensim dictionary from the texts
dictionary = gensim.corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

time: 877 ms


In [59]:
corpus[0][:10]

[(94, 1),
 (101, 1),
 (152, 1),
 (158, 1),
 (165, 2),
 (199, 1),
 (361, 1),
 (364, 1),
 (383, 1),
 (437, 1)]

time: 417 ms


In [60]:
#gensim.models.LdaMulticore
lda = gensim.models.LdaMulticore(corpus, num_topics=10, passes=10,
                            id2word=dictionary)

time: 47.3 s


In [61]:
lda.show_topics(num_words=20)


[u"0.009*would + 0.007*like + 0.007*Windows + 0.007*n't + 0.007*use + 0.005*problem + 0.005*good + 0.005*'s + 0.005*What + 0.004*one + 0.004*This + 0.004*network + 0.004*files + 0.003*How + 0.003*'ve + 0.003*'m + 0.003*computer + 0.003*open + 0.003*Microsoft + 0.003*user",
 u"0.013*n't + 0.012*Windows + 0.009*Is + 0.006*use + 0.006*would + 0.006*way + 0.006*'m + 0.005*know + 0.005*folder + 0.005*The + 0.005*XP + 0.005*windows + 0.005*using + 0.004*like + 0.004*What + 0.004*get + 0.004*files + 0.004*new + 0.004*need + 0.004*password",
 u"0.028*r + 0.027*None + 0.014*dc + 0.014*y + 0.013*tc + 0.013*PM + 0.013*0x0-0x1316315 + 0.013*.com.google.Chrome + 0.004*gt + 0.004*lt + 0.003*system + 0.003*GoogleSoftwareUpdateAgent + 0.003*Section + 0.003*using + 0.003*EndSection + 0.003*n't + 0.003*92673/0xb00a1000 + 0.003*lvl=2 + 0.003*Option + 0.003*Identifier",
 u"0.013*'m + 0.013*Windows + 0.012*'s + 0.010*n't + 0.009*use + 0.008*Is + 0.007*would + 0.007*like + 0.006*one + 0.006*The + 0.006*way 

time: 6.13 ms


In [62]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['would', 'like', 'Windows', "n't", 'use', 'problem', 'good', "'s", 'What', 'one', 'This', 'network', 'files', 'How', "'ve", "'m", 'computer', 'open', 'Microsoft', 'user']
()
["n't", 'Windows', 'Is', 'use', 'would', 'way', "'m", 'know', 'folder', 'The', 'XP', 'windows', 'using', 'like', 'What', 'get', 'files', 'new', 'need', 'password']
()
['r', 'None', 'dc', 'y', 'tc', 'PM', '0x0-0x1316315', '.com.google.Chrome', 'gt', 'lt', 'system', 'GoogleSoftwareUpdateAgent', 'Section', 'using', 'EndSection', "n't", '92673/0xb00a1000', 'lvl=2', 'Option', 'Identifier']
()
["'m", 'Windows', "'s", "n't", 'use', 'Is', 'would', 'like', 'one', 'The', 'way', 'time', 'computer', 'monitor', 'get', "'ve", 'What', 'work', 'XP', 'want']
()
["n't", "'m", 'drive', 'like', 'use', 'Windows', "'s", "'ve", 'Is', 'iPhone', 'The', 'What', 'Vista', 'using', 'It', 'computer', 'could', 'video', 'one', 'would']
()
["n't", 'would', 'Windows', 'file', "'s", 'get', 'work', 'like', 'It', 'Is', 'one', 'r-xp', 'use', 'The', 'f